In [1]:
using Laplacians

In [2]:
pwd()

"/Users/serbanstan/git/Laplacians.jl/debug/Solver Testing"

In [3]:
include("../../src/isotonicIPM.jl");

In [4]:
solvers = [augTreeSolver, KMPSDDSolver, samplingSDDSolver, AMGSolver]

4-element Array{Function,1}:
 Laplacians.augTreeSolver    
 Laplacians.KMPSDDSolver     
 Laplacians.samplingSDDSolver
 Laplacians.AMGSolver        

In [5]:
function generateInput(a)
    n = a.n
    
    b = zeros(Float64,n);
    b[1] = 1;
    b[n] = -1;
    f = KMPLapSolver(a, verbose=false, tol=1e-1); # use any other solver
    v = f(b)
    v = v / (v[1] - v[n])

    p = sortperm(v);
    ap = symPermuteCSC(a, p);
    # vp = v[p]
    vp = v[p] + rand(n) / sqrt(n);
    at = triu(ap);
    
    return at,vp
end

generateInput (generic function with 1 method)

In [6]:
n = 10000

nlist = []
ilist = []
edges = []
targetTol = []

iters = [Array{Float64,1}(0) for s in solvers]
precision = [Array{Float64,1}(0) for s in solvers]
times = [Array{Float64,1}(0) for s in solvers]

for i in 1:200
    a = wtedChimera(n,i)
    a,v = generateInput(a);
    
    push!(nlist, a.n)
    push!(ilist, i)
    push!(edges, nnz(a))
    push!(targetTol, 1e-5)
    
    for si in 1:length(solvers)
        solver = solvers[si]
        
        try
            mysolver = (H -> solver(H,tol=1e-1,maxits=1000,maxtime=20))
            
            tic()
            x,acc,itercount = isotonicIPMrelEps(a,v,1e-5,mysolver)
            push!(times[si], toc())

            push!(iters[si], itercount)
            push!(precision[si], acc)
        catch
            push!(times[si], Inf)
            push!(iters[si], Inf)
            push!(precision[si], Inf)
            
            println("Error for ", i, " and ", solver)
        end
    end
    
    println("Done for ", i)
end

elapsed time: 28.288614889 seconds
elapsed time: 24.121227419 seconds
elapsed time: 41.758927382 seconds
elapsed time: 33.783402356 seconds
Done for 1
elapsed time: 13.101008912 seconds
elapsed time: 19.410475539 seconds
elapsed time: 31.965511861 seconds
elapsed time: 23.257051724 seconds
Done for 2


In [7]:
tab = ["nlist"; nlist]
tab = [tab ["ilist"; ilist]]
tab = [tab ["edges"; edges]]
tab = [tab ["targetTol"; targetTol]]

for i in 1:length(solvers)
    name = split(string(solvers[i]), ".")[2]

    
    tab = [tab [name * "_runtime"; times[i]]]
    tab = [tab [name * "_iterations"; iters[i]]]
    tab = [tab [name * "_convergence"; precision[i]]]
end

In [8]:
fn = string("isotonicIPMResults.csv");
writecsv(fn,tab);